In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl 
import torchmetrics
from torchmetrics import Metric

some useful methods in pl.LightningModule
* print
* log
* forward
* training_step
* training_step_end
* training_epoch_end
* validation_step
* test_step
* predict_step
* configure_optimizers

In [2]:
class MyAccuracy(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        preds = torch.argmax(preds, dim=1)
        assert preds.shape == target.shape
        self.correct += torch.sum(preds == target)
        self.total += len(target)
        
    def compute(self):
        return self.correct.float() / self.total    

class NN(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(task='multiclass', num_classes=num_classes, average='macro')
        self.my_accuracy = MyAccuracy()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _common_step(self, batch, batch_idx):
        x, y = batch 
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        return loss, scores, y

        
    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        acc = self.accuracy(scores, y)
        myacc = self.my_accuracy(scores, y)
        f1_score = self.f1_score(scores, y)
        self.log_dict({'train_loss':loss, 'train_accuracy':acc, 'train_myaccuracy':myacc, 'train_f1_score':f1_score}, 
                      on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return {'loss':loss, 'scores':scores, 'y':y}
    
    
    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log('val_loss', loss, sync_dist=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log('test_loss', loss, sync_dist=True)
        return loss

    def predict_step(self, batch, batch_idx):
        x, y = batch 
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [3]:
# Initialize network
input_size = 784
num_classes = 10
model = NN(input_size=input_size, num_classes=num_classes)


In [4]:

# batch_size = 64
# # Load Data
# entire_dataset = datasets.MNIST(
#     root="/data/", train=True, transform=transforms.ToTensor(), download=True
# )
# train_ds, val_ds = random_split(entire_dataset, [50000, 10000])
# test_ds = datasets.MNIST(
#     root="/data/", train=False, transform=transforms.ToTensor(), download=True
# )
# train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)


class MnistDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
    
    def prepare_data(self):
        # prepare the raw data
        datasets.MNIST(root=self.data_dir, train=True, download=True)
        datasets.MNIST(root=self.data_dir, train=False, download=True)
        
    def setup(self, stage):
        if stage == 'fit':
            entire_dataset = datasets.MNIST(root=self.data_dir, 
                                            train=True, 
                                            transform=transforms.ToTensor(), 
                                            download=False)
        
            self.train_ds, self.val_ds = random_split(entire_dataset, [50000, 10000], generator=torch.Generator().manual_seed(123))
        
        if stage == 'test':
            self.test_ds = datasets.MNIST(root=self.data_dir, 
                                    train=False, 
                                    transform=transforms.ToTensor(), 
                                    download=False)
    def train_dataloader(self):
        return DataLoader(dataset=self.train_ds, 
                          batch_size=self.batch_size, 
                          num_workers=self.num_workers, 
                          shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(dataset=self.val_ds, 
                          batch_size=self.batch_size, 
                          num_workers=self.num_workers, 
                          shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(dataset=self.test_ds, 
                          batch_size=self.batch_size, 
                          num_workers=self.num_workers, 
                          shuffle=False)

dm = MnistDataModule(data_dir="/data/", batch_size=64, num_workers=1)

In [5]:
trainer = pl.Trainer(accelerator='gpu', devices=[0],  # cannot do 2 GPUs in ipynb
                     min_epochs=1, max_epochs=3, precision='16-mixed')


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [6]:
trainer.fit(model, dm)

You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]

  | Name        | Type               | Params
---------------------------------------------------
0 | fc1         | Linear             | 39.2 K
1 | fc2         | Linear             | 510   
2 | loss_fn     | CrossEntropyLoss   | 0     
3 | accuracy    | MulticlassAccuracy | 0     
4 | f1_score    | MulticlassF1Score  | 0     
5 | my_accuracy | MyAccuracy         | 0     
---------------------------------------------------
39.8 K    Trainable params
0         Non-trainable params
39.8 K    Total params
0.159     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [7]:
trainer.validate(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            0.1712319403886795
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.1712319403886795}]

In [8]:
trainer.test(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]
/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.16148872673511505
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.16148872673511505}]

In [9]:
dm.train_dataloader()